In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [6]:
years = [str(year) for year in range(2011,2023)]

In [7]:
contest_links = {}
for y in years:
    response = requests.get('https://contests.npcnewsonline.com/contests/' + y + '/npc')
    content = response.content
    parser = BeautifulSoup(content,'html.parser')
    contests = parser.find('div', class_= 'contest-listing').find_all('a')
    for c in contests:
        contest_links[y] = contest_links.get(y,[]) + [c.get('href')]

In [8]:
from tqdm import tqdm

In [10]:
df = pd.DataFrame(columns=['competitor_name', 'contest_name', 'contest_date', 'division', 'class', 'placing'])
for y in tqdm(years):
    for link in tqdm(contest_links[y]):
        response = requests.get(link)
        content = response.content
        parser = BeautifulSoup(content, 'html.parser')
        contest_name = parser.find(class_='entry-title').text
        contest_date = parser.find(class_='entry-date').text
        contest_table = parser.find('table', class_='contest_table')
        for x in contest_table.find_all('td'):
            division = x.find('h2')
            classes = x.find_all('div')

            
            classes_text = [c.text.replace(' ','-') for c in classes]
            for c in classes_text:
                competitors = x.find_all(attrs={'data-parent':c,'data-person':'yes'})
                for p in competitors:
                    if p.text.find('Comparisons') != -1:
                        continue
                    placing = p.find('span').text
                    if placing:
                        name = p.text[len(placing):]
                    df.loc[len(df)] = [name, contest_name, contest_date, division.text, c, placing]

100%|███████████████████████████████████████| 12/12 [6:53:56<00:00, 2069.74s/it]


In [11]:
df.head()

,competitor_name,contest_name,contest_date,division,class,placing
0,Kevin Jordan,NPC Junior Nationals,"June 17, 2011",MEN'S BODYBUILDING,overall,
1,Anthony Tenuta,NPC Junior Nationals,"June 17, 2011",MEN'S BODYBUILDING,super-heavyweight,1
2,Kevin Tomasini,NPC Junior Nationals,"June 17, 2011",MEN'S BODYBUILDING,super-heavyweight,2
3,Daniel Massey,NPC Junior Nationals,"June 17, 2011",MEN'S BODYBUILDING,super-heavyweight,3
4,Timmy Gaillard,NPC Junior Nationals,"June 17, 2011",MEN'S BODYBUILDING,super-heavyweight,4


In [12]:
df.info

<bound method DataFrame.info of           competitor_name                       contest_name     contest_date  \
0           Kevin Jordan                NPC Junior Nationals    June 17, 2011   
1         Anthony Tenuta                NPC Junior Nationals    June 17, 2011   
2         Kevin Tomasini                NPC Junior Nationals    June 17, 2011   
3          Daniel Massey                NPC Junior Nationals    June 17, 2011   
4         Timmy Gaillard                NPC Junior Nationals    June 17, 2011   
...                   ...                                ...              ...   
565211       Tara Klaker   NPC Wisconsin State Championships  October 6, 2022   
565212       Tara Klaker   NPC Wisconsin State Championships  October 6, 2022   
565213      Nevaeh Grant   NPC Wisconsin State Championships  October 6, 2022   
565214      Aimee Preece   NPC Wisconsin State Championships  October 6, 2022   
565215     Jennife Labbe   NPC Wisconsin State Championships  October 6, 2022

In [14]:
df.to_csv('npc_contest_data_06_01_2023.csv')